## a. Implementacja binarnej reprezentacji chromosomu + konfiguracja dokładności
- `bits` - liczba bitów chromosomu, liczba całkowita

In [77]:
import random

In [88]:
def gen_chromosome(bits):
    return [random.randrange(0, 2) for row in range(bits)]

In [89]:
print(gen_chromosome(9))

[1, 0, 0, 0, 1, 0, 0, 1, 0]


## Implementacja konfiguracji wielkości osobnika
- `bits` - liczba bitów chromosomu, liczba całkowita
- `chromoQuantity` - liczba chromosomów w każdym osobniku, liczba całkowita

In [98]:
def get_individual(bits, chromoQuantity):
    return [gen_chromosome(bits) for row in range(chromoQuantity)]

In [97]:
print(get_individual(9, 2))

[[1, 1, 0, 0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 0, 1, 1, 0, 1]]


## b. Implementacja konfiguracji wielkości populacji
- `bits` - liczba bitów chromosomu, liczba całkowita
- `chromoQuantity` - liczba chromosomów w każdym osobniku, liczba całkowita
- `popSize` - liczba chromosomów w populacji, liczba całkowita

In [99]:
def gen_population(bits, chromoQuantity, popSize):
    return [get_individual(bits, chromoQuantity) for row in range(popSize)]

In [106]:
population = gen_population(9, 2, 20)
population

[[[0, 0, 1, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 1, 1, 0]],
 [[0, 0, 0, 1, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 0, 0, 1, 1]],
 [[1, 1, 0, 1, 0, 1, 0, 0, 1], [0, 1, 1, 0, 0, 0, 1, 0, 0]],
 [[1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0, 1, 1]],
 [[0, 0, 0, 0, 0, 0, 0, 1, 1], [1, 0, 0, 1, 0, 1, 1, 1, 1]],
 [[1, 1, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 1, 1, 0, 1, 1, 1]],
 [[1, 0, 1, 0, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0]],
 [[0, 0, 1, 1, 1, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 0, 1, 0]],
 [[0, 0, 1, 0, 1, 1, 1, 1, 0], [0, 0, 0, 1, 1, 0, 0, 1, 0]],
 [[1, 1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 1, 1, 0]],
 [[0, 1, 1, 1, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 0, 1, 0]],
 [[0, 1, 0, 1, 0, 1, 1, 1, 1], [0, 1, 0, 0, 1, 0, 1, 1, 0]],
 [[1, 0, 1, 0, 0, 0, 0, 1, 1], [1, 1, 0, 1, 0, 0, 0, 1, 1]],
 [[0, 1, 1, 1, 1, 0, 1, 1, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1]],
 [[0, 0, 1, 1, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0]],
 [[1, 0, 1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 1, 0, 1]],
 [[1, 0, 0, 0, 1, 0, 1, 

## Implementacja rozkodowania osobników - konwersji osobnika z reprezentacji binarnej do reprezentacji dziesiętnej
- `binary` - osobnik w reprezentacji binarnej, tablica jednowymiarowa `ndarray`
- `decimal` - osobnik w reprezentacji dziesiętnej, liczba całkowita

In [62]:
def toDecimal(binary):
    decimal = 0
    position = len(binary)
    for bit in binary:
        position -= 1
        decimal += bit * pow(2, position)
    return decimal

In [128]:
toDecimal(population[0][0])

100

## Implementacja konwersji populacji z reprezentacji binarnej do dziesiętnej 
- `population` - populacja osobników, lista

In [125]:
def popToDecimal(population):
    return [[toDecimal(chromo) for chromo in individual] for individual in population]

In [129]:
decimalPop = popToDecimal(population)
decimalPop

[[100, 30],
 [47, 195],
 [425, 196],
 [448, 451],
 [3, 303],
 [454, 183],
 [348, 112],
 [118, 66],
 [94, 50],
 [400, 358],
 [240, 90],
 [175, 150],
 [323, 419],
 [246, 81],
 [125, 504],
 [341, 333],
 [279, 420],
 [476, 146],
 [60, 418],
 [472, 34]]

## c. Implementacja konfiguracji liczby epok

## Implementacja funkcji Booth dwóch zmienych
- `x` - pierwsza zmienna, liczba całkowita
- `y` - druga zmienna, liczba całkowita

In [64]:
def boothFunc(x, y):
    return pow((x + 2 * y - 7), 2) + pow((2 * x + y - 5), 2)

## Implementacja funkcji Booth całej populacji
- `decimalPop` - populacja osobników w reprezentacji dziesiętnej, lista

In [132]:
def popBoothFunc(decimalPop):
    return [boothFunc(individual[0], individual[1]) for individual in decimalPop]

In [279]:
popAdaptation = popBoothFunc(decimalPop)
popAdaptation

[74034,
 265556,
 1739781,
 3604613,
 454820,
 1840365,
 964034,
 147258,
 89258,
 2559290,
 489794,
 464049,
 2455316,
 483425,
 1828877,
 2020100,
 2183273,
 1773770,
 1074410,
 1230818]

## d. Implementacja selekcji metodą najlepszych
- `popAdaptation` - funkcje przystosowania populacji, lista
- `percent` - procent najlepszych osobników, liczba całkowita
- `bestSelected` - funkcje przystosowania najlepszych osobników, lista

In [299]:
def bestSelection(popAdaptation, percent = 30):
    bestSelected = []
    N = int(len(popAdaptation) * percent / 100)
    for i in range(0, N):  
        best = max(popAdaptation)
        popAdaptation.remove(best); 
        bestSelected.append(best)
    return bestSelected

In [300]:
bestSelected = bestSelection(popAdaptation[:], 10)
bestSelected

[3604613, 2559290]

## d. Implementacja selekcji metodą ruletki
